# OpenCV中尋找輪廓
### `cv2.findContours(image, mode, method, contours=None, offset=None)`

* image：輸入圖像，必須是二值圖像，通常是經過閾值處理或邊緣檢測後的結果。
* mode：輪廓檢測模式，這個參數決定輪廓的檢索方式：
    * cv2.RETR_EXTERNAL：只檢索最外層的輪廓。
    * cv2.RETR_LIST：檢索所有輪廓，並將其放入列表。
    * cv2.RETR_CCOMP：檢索所有輪廓並將它們組織成兩層結構：外層是物體的外圍邊界，內層是空洞的邊界。
    * cv2.RETR_TREE：檢索所有輪廓並創建一個完整的家族層次結構。
* method：輪廓逼近方法，它決定如何儲存輪廓的點：
    * cv2.CHAIN_APPROX_NONE：儲存輪廓上所有的點。
    * cv2.CHAIN_APPROX_SIMPLE：壓縮水平、垂直和對角線方向的輪廓，只保留端點。
    * cv2.CHAIN_APPROX_TC89_L1 或 cv2.CHAIN_APPROX_TC89_KCOS：使用Teh-Chin鏈逼近算法的一種。
* contours：一個 Python 列表，用於儲存檢測到的輪廓。
* offset：可選，用於對輪廓點進行偏移。

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# display matplotlib plots directly within the notebook interface. especially when working in the Jupyter Notebook
%matplotlib inline

In [ ]:
img = cv2.imread('data/5.png')
print(img.shape)
imageBlack = np.zeros((img.shape[0], img.shape[1]))
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

ret,thresh = cv2.threshold(gray,0,255,1)
plt.axis('off')
plt.imshow(thresh, 'gray')
plt.show()

kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5, 5))     
dilated = cv2.dilate(thresh,kernel)

contours, hierarchy = cv2.findContours(dilated,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(imageBlack,contours,-1,(255,255,255),10) 

plt.axis('off')
plt.imshow(imageBlack, 'gray')
plt.show()
boxes = []
print("hierarchy:\n",hierarchy)
print("hierarchy.shape =",hierarchy.shape)
boxes.append(hierarchy[0][1])
x,y,w,h = cv2.boundingRect(contours[1])
imgrgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
cv2.drawContours(imgrgb, contours, 0, (0,255,0), 10)
imgrgb = cv2.rectangle(imgrgb,(x-1,y-1),(x+w+1,y+h+1),(0,0,255),10)

plt.axis('off')
plt.imshow(imgrgb)
plt.show()

### [練習]Tryitnow 輪廓提取
![](data/TIN_1.png)
![](data/TIN_2.png)

In [ ]:
import cv2
import numpy as np
from skimage.morphology import skeletonize
import matplotlib.pyplot as plt
%matplotlib inline

def paintContour(curr, mode):
        #x,y,w,h = cv2.boundingRect(contours[curr])
        #imgrgb = cv2.rectangle(imgrgb,(x-1,y-1),(x+w+1,y+h+1),(0,0,255),10)
        if mode == 0:
            cv2.drawContours(imageBlack,contours,curr,(255,255,255),-1)
        else:
            cv2.drawContours(imageBlack,contours,curr,(0,0,0),-1)

def show(image_show):
    plt.axis('off')
    plt.imshow(image_show, 'gray')
    plt.show()
            
img = cv2.imread('data/tryitnow.png')
print(img.shape)
imageBlack = np.zeros(img.shape)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)#????
show(gray)

ret,thresh = cv2.threshold(gray,150,255,1)#????
show(thresh)

contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)#?????
print(hierarchy)
#找出所有父輪廓為0點輪廓

imgrgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
imageBlack = np.zeros(img.shape, np.uint8)
print(hierarchy)
for i in range(len(hierarchy[0])):
    print(i,hierarchy[0,i,3])
    if hierarchy[0,i,3] == 0:
        cv2.drawContours(imgrgb,contours,i,(0,0,255),20)
        show(imgrgb)
        
parent = hierarchy[0][0]
current = parent[2]
blacks = []
while current != -1:
    paintContour(current, 1)
    current = hierarchy[0][current][0]
    nxt = hierarchy[0][current][2]
    if nxt != -1:
        blacks.append(nxt)
for b in blacks:
    paintContour(current, 0)

# OpenCV中的輪廓近似
### 基礎框選
* `cv2.boundingRect(cnt)`<br>
計算輪廓的直立邊界矩形，即沒有旋轉的最小矩形
* `cv2.minAreaRect()` <br>
計算輪廓的最小有向邊界矩形，矩形可能會有旋轉
* `cv2.minEnclosingCircle()` <br>
計算完全覆蓋輪廓的最小圓的中心點和半徑
### 輪廓本身的特徵分析
* 使用 `cv2.moments(cnt)`<br>
 計算輪廓的矩，可以獲取質心、面積等資訊
* 使用 `cv2.contourArea(cnt)`<br>
  直接計算輪廓的面積
* 使用 `cv2.arcLength(cnt,True)`<br>
  計算輪廓的周長，可指定輪廓是否閉合
* 使用`cv2.approxPolyDP(cnt,epsilon,True) `<br>
可以透過調整準確度引數來控制輪廓近似的精準度<br>
epsilon=0.1*cv2.arcLength()
* 使用`cv2.convexHull(points,clockwise,returnPoints)`<br>
物體輪廓突出檢測和糾正輪廓的凸性缺陷，通常用於確定圖像輪廓的外形


In [ ]:
import cv2
import numpy as np

"""
REFER: https://github.com/Yonv1943/Python/blob/master/Demo/TUTO_edge_detection.py
2024-05-02 opencv4.9 
"""
def draw_contours(img, cnts):  # conts = contours
    img = np.copy(img)
    img = cv2.drawContours(img, cnts, -1, (0, 255, 0), 2)
    return img

def draw_min_rect_circle(img, cnts):  # conts = contours
    img = np.copy(img)

    for cnt in cnts:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)  # blue

        min_rect = cv2.minAreaRect(cnt)  # min_area_rectangle
        min_rect = np.intp(cv2.boxPoints(min_rect))
        cv2.drawContours(img, [min_rect], 0, (0, 255, 0), 2)  # green

        (x, y), radius = cv2.minEnclosingCircle(cnt)
        center, radius = (int(x), int(y)), int(radius)  # center and radius of minimum enclosing circle
        img = cv2.circle(img, center, radius, (0, 0, 255), 2)  # red
    return img


def draw_approx_hull_polygon(img, cnts):
    # img = np.copy(img)
    img = np.zeros(img.shape, dtype=np.uint8)

    cv2.drawContours(img, cnts, -1, (255, 0, 0), 2)  # blue

    min_side_len = img.shape[0] / 32  # 多边形边长的最小值 the minimum side length of polygon
    min_poly_len = img.shape[0] / 16  # 多边形周长的最小值 the minimum round length of polygon
    min_side_num = 3  # 多边形边数的最小值
    approxs = [cv2.approxPolyDP(cnt, min_side_len, True) for cnt in cnts]  # 以最小边长为限制画出多边形
    approxs = [approx for approx in approxs if cv2.arcLength(approx, True) > min_poly_len]  # 筛选出周长大于 min_poly_len 的多边形
    approxs = [approx for approx in approxs if len(approx) > min_side_num]  # 筛选出边长数大于 min_side_num 的多边形
    # Above codes are written separately for the convenience of presentation.
    cv2.polylines(img, approxs, True, (0, 255, 0), 2)  # green

    hulls = [cv2.convexHull(cnt) for cnt in cnts]
    cv2.polylines(img, hulls, True, (0, 0, 255), 2)  # red

    # for cnt in cnts:
    #     cv2.drawContours(img, [cnt, ], -1, (255, 0, 0), 2)  # blue
    #
    #     epsilon = 0.02 * cv2.arcLength(cnt, True)
    #     approx = cv2.approxPolyDP(cnt, epsilon, True)
    #     cv2.polylines(img, [approx, ], True, (0, 255, 0), 2)  # green
    #
    #     hull = cv2.convexHull(cnt)
    #     cv2.polylines(img, [hull, ], True, (0, 0, 255), 2)  # red
    return img



image = cv2.imread('data/test.png')  # a black objects on white image is better

# gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
# ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
thresh = cv2.Canny(image, 128, 256)

contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# print(hierarchy, ":hierarchy")
"""
[[[-1 -1 -1 -1]]] :hierarchy  # cv2.Canny()

[[[ 1 -1 -1 -1]
  [ 2  0 -1 -1]
  [ 3  1 -1 -1]
  [-1  2 -1 -1]]] :hierarchy  # cv2.threshold()
"""

imgs = [
    image, thresh,
    draw_min_rect_circle(image, contours),
    draw_approx_hull_polygon(image, contours),
]

for img in imgs:
    # cv2.imwrite("%s.jpg" % id(img), img)
    cv2.imshow("contours", img)
    cv2.waitKey(1500)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

"""
參考: https://github.com/Yonv1943/Python/blob/master/Demo/TUTO_edge_detection.py
2024-05-02 opencv4.9 
"""
def draw_contours(img, cnts):  # cnts = contours
    img = np.copy(img)
    img = cv2.drawContours(img, cnts, -1, (0, 255, 0), 2)
    return img

def draw_min_rect_circle(img, cnts):  # cnts = contours
    img = np.copy(img)

    for cnt in cnts:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)  # 藍色

        min_rect = cv2.minAreaRect(cnt)  # 最小面積矩形
        min_rect = np.intp(cv2.boxPoints(min_rect))
        cv2.drawContours(img, [min_rect], 0, (0, 255, 0), 2)  # 綠色

        (x, y), radius = cv2.minEnclosingCircle(cnt)
        center, radius = (int(x), int(y)), int(radius)  # 最小外接圓的中心和半徑
        img = cv2.circle(img, center, radius, (0, 0, 255), 2)  # 紅色
    return img


def draw_approx_hull_polygon(img, cnts):
    img = np.zeros(img.shape, dtype=np.uint8)
    for cnt in cnts:
        cv2.drawContours(img, [cnt, ], -1, (255, 0, 0), 2)  # 藍

        # arcLength計算輪廓的周長，True 表示輪廓是閉合的
        epsilon = 0.02 * cv2.arcLength(cnt, True) 
        # approxPolyDP進行輪廓近似，epsilon 是近似的精確度，較小的 epsilon 值導致輪廓與原始輪廓更為接近
        approx = cv2.approxPolyDP(cnt, epsilon, True) 
        cv2.polylines(img, [approx, ], True, (0, 255, 0), 2)  # 綠
        
        # convexHull獲取輪廓的凸包，凸包是包含輪廓的最小凸多邊形
        hull = cv2.convexHull(cnt)
        cv2.polylines(img, [hull, ], True, (0, 0, 255), 2)  # 紅

    return img


image = cv2.imread('data/test.png')  # 黑物體在白背景下效果更佳
thresh = cv2.Canny(image, 128, 256)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

imgs = [
    image, thresh,
    draw_min_rect_circle(image, contours),
    draw_approx_hull_polygon(image, contours),
]

for img in imgs:
    cv2.imshow("contours", img)
    cv2.waitKey(2000)
cv2.destroyAllWindows()

# OpenCV中的直線偵測
#### `cv2.HoughLinesP(image, rho, theta, threshold, lines=None, minLineLength=None, maxLineGap=None) `

* image：輸入的應該是邊緣檢測後的二進制圖像
* rho：距離解析度，單位為像素;theta：角度解析度，通常為 numpy.pi / 180
* threshold：閾值，決定最小票數（累積平面的票數）來檢測線段
* minLineLength：線段的最小長度，線段短於此長度的將被拒絕
* maxLineGap：同一方向上兩個線段之間的最大間隙
如果間隙較大，將被視為兩條獨立的線段

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

img = cv2.imread('data/chessboard.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 100, 255)
plt.axis("off")
plt.imshow(edges, "gray")
plt.show()

lines = cv2.HoughLinesP(edges,1,np.pi/180,0)

for points in lines:
    x1 = points[0][0]
    y1 = points[0][1]
    x2 = points[0][2]
    y2 = points[0][3]
    cv2.line(img,(x1,y1),(x2,y2),(255,0,0),5)
    
plt.axis("off")
plt.imshow(img)
plt.show()


# OpenCV中的圓形偵測
##### `cv2.HoughCircles(image, method, dp, minDist, param1=None, param2=None, minRadius=None, maxRadius=None)`
##### 參數說明
* image: 輸入圖像，必須是灰階圖像。
* method: 檢測方法，目前 OpenCV 只實現了 cv.HOUGH_GRADIENT 方法。
* dp: 累加器解析度與圖像解析度的比值。如果 dp=1，累加器的解析度與輸入圖像相同。如果 dp=2，累加器的解析度是輸入圖像的一半。
* minDist: 檢測到的圓的中心（x, y）坐標之間的最小距離。這個參數可以避免多個鄰近圓被錯誤地檢測為同一個圓。
* param1: 第一個方法特定的參數。在使用 HOUGH_GRADIENT 方法時，這個參數表示用於邊緣檢測的 Canny 高閾值（低閾值是高閾值的一半）。
* param2: 第二個方法特定的參數。在 HOUGH_GRADIENT 方法中，它是霍夫圓變換的累加器閾值。只有那些累積器中的值高於此閾值的圓才會被考慮。這個參數越小，檢測到的圓就越多。
* minRadius: 圓半徑的最小值。
* maxRadius: 圓半徑的最大值。

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
img = cv2.imread('data/eye.jpg')#eye.jpgcoins.jpg
img = cv2.medianBlur(img,5)
cimg = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

print(cimg.shape)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
print(gray.shape)
#circles = cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT,1,70,param1=30,param2=10,minRadius=30,maxRadius=49)
circles = cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT,1,39,param1=50,param2=15,minRadius=140,maxRadius=150)
circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    #畫外圓
    cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
    #畫圓中心
    cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)

plt.imshow(cimg)
plt.title('detected circles')
plt.show()

### [練習] 
##### 請找出10元的硬幣有幾個
##### 設計一自動識別硬幣金額的程式 ANS:250元

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
img = cv2.imread('data/coins.jpg')
img = cv2.medianBlur(img,5)
cimg = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

print(cimg.shape)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
print(gray.shape)
circles = cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT,1,70,param1=30,param2=10,minRadius=30,maxRadius=49)
circles = np.uint16(np.around(circles))
MoneyCount = 0
for i in circles[0,:]:
    print(i[0],i[1])
    #print(cimg[i[1],i[0]])
    if i[2]>42 and (cimg[i[1],i[0]][0]-cimg[i[1],i[0]][2]>40):
        cv2.circle(cimg,(i[0],i[1]),i[2],(255,0,0),3)
        MoneyCount+=50
    elif i[2]<36 and (cimg[i[1],i[0]][0]-cimg[i[1],i[0]][2]>40):
        cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),3)
        MoneyCount+=1
    elif i[2]<39:
        MoneyCount+=5
        cv2.circle(cimg,(i[0],i[1]),i[2],(0,0,255),3)
    else:
        MoneyCount+=10
        cv2.circle(cimg,(i[0],i[1]),i[2],(0,0,0),3)
print("Total Money:",MoneyCount)
plt.imshow(cimg)
plt.title('detected circles')
plt.show()

# OpenCV中的Harris角點偵測
### `角點` 向任何方向移動變化都很大的點
cv2.cornerHarris(src, blockSize, ksize, k) 
* img - 資料型別為float32 的輸入影像
* blockSize - 角點檢測中要考慮的領域大小
* ksize - Sobel 求導中使用的視窗大小
* k - Harris 角點檢測方程中的自由引數，取值引數為[0,04，0.06]

In [ ]:
import numpy as np
import cv2 as cv

filename = 'data/chessboard.png'
img = cv.imread(filename)
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

gray = np.float32(gray)
dst = cv.cornerHarris(gray,3,3,0.04)

#result is dilated for marking the corners, not important
dst = cv.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]

cv.imshow('dst',img)
cv.waitKey(0)
cv.destroyAllWindows()

### Shi-Tomasi 角點檢測算法
```
對 Harris 角點檢測算法的修改，通常提供更好的結果。<br>
它在涉及特徵跟蹤和運動分析的計算機視覺任務中特別有用。<br>
```
corners = cv.goodFeaturesToTrack(gray, maxCorners, qualityLevel, minDistance)
##### 參數說明：
* gray：第一個參數是影像，需要是灰階圖像（cv2.CV_8U 或 cv2.CV_32F 類型）。
* MaxCorners：這是函式將返回的角點的最大數量。如果找到的角點多於此數量，它將根據最小質量閾值返回最強的角點。將此參數設置為零意味著沒有最大限制（將返回所有檢測到的角點）。
* QualityLevel：此參數指定低於該質量的角點將被拒絕。該值乘以最佳角點質量度量（通常是最小特徵值，即 Shi-Tomasi 分數）。典型值可能在 0.01 到 0.1 範圍內。
* MinDistance：這是返回角點之間可能的最小歐幾里得距離。即確保所有返回的角點至少相隔這麼多像素。這在避免由於角點檢測過程中的小錯誤或噪聲而在近距離內多次檢測角點時特別有用。
##### 額外參數：
* Mask (可選)：一個可選的感興趣區域。如果指定，算法只在此區域內尋找角點。
* BlockSize (可選)：用於計算每個像素鄰域上導數協方差矩陣的平均塊大小。典型值為 3, 5, 或 7。
* GradientSize (可選)：用於找尋圖像梯度的 Sobel 核大小。
* UseHarrisDetector (可選)：布爾標誌，指示是否使用 Harris 檢測器或 Shi-Tomasi 方法。

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

img = cv.imread('data/ChessBoard.png')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

corners = cv.goodFeaturesToTrack(gray,0,0.01,10)
corners = np.intp(corners)

for i in corners:
    x,y = i.ravel()
    cv.circle(img,(x,y),10,255,-1)

plt.imshow(img),plt.show()